***WMC via Knowledge compilation***


---


1. Write a method that transforms a formula in sd-DNNF form, and use
this method for computing the weighted model count of propositional
formulas.
2. Check the correctness of your algorithm by comparing the results of
your method with the explicit computation of weighted model
counting via truth table.
3. Write a method that estimates #SAT using the sampleSat algorithm,
and compare the result of the approximated counting with the result
obtained by the exact counting.






In [1]:
from sympy import symbols, simplify, Not, And, Or, Implies, satisfiable
from itertools import combinations
from functools import reduce
#libreria per il debugging
#import pdb
#%pdb on


Pipeline to convert a formula F in sd-DNNF:

1. prendiamo una formula ***F***.
2. trasformo in ***NNF*** : formula in cui le negazioni compaiono solo davanti a preposizioni atomiche.
3. trasformo in ***DNNF*** (*Decomposable Negation Normal Form*) : è una NNF in cui ogni conjunction non ha proposizioni atomiche in comune, cioè `Φ₁ ∧ Φ₂` tale che `props(Φ₁) ∩ (Φ₂) = ∅` e per farlo applico Shannon's Expansion.
4. trasformo in ***d-DNNF*** (*Deterministic Decomposable Negation Normal Form*) : è una DNNF  in cui per ogni disjunction `Φ₁ v Φ₂ v .. v Φₙ` presenti nella formula stessa, c'è al massimo una i tale che `I |= Φᵢ`
Questo perchè i rami di un OR devono essere mutuamente esclusivi, altrimenti nel WMC rischiamo di contare due volte la stessa assegnazione di verità.
5. trasformo in ***sd-DNNF*** (*Smooth Deterministic Decomposable Negation Normal Form*) : applico smoothing left and right per garantire che tutti i disgiunti contengano tutte le variabili proposizionali - vedi regole di trasformazione (in sostanza viene usata una tautologia per le proposizioni atomiche mancanti)


Nota Bene: In realtà, il primissimo passo sarebbe la trasformazione di F in ***CNF***: è una formula in cui non compaiono i simboli di implicazione (→) o equivalenza (≡), ma nel nostro codice viene fatto dalla funzione to_nnf (importata).

In [2]:
# ============================================================
#  F -> NNF -> DNNF -> d-DNNF -> sd-DNNF (con Tau) -> WMC
#  (niente normalize_formula: usiamo direttamente to_nnf)
# ============================================================

from sympy import symbols, And, Or, Not, S, simplify, sympify
from sympy.logic.boolalg import to_nnf, Boolean
from sympy.logic.inference import satisfiable

# ---------------------------
# Nodo booleano Tau(v) per smoothing strutturale visibile
# ---------------------------
class Tau(Boolean):
    """
    Rappresenta la tautologia (v ∨ ¬v) come nodo esplicito.
    Nel WMC contribuisce con [w(v) + w(¬v)].
    """
    is_Tau = True

    def __new__(cls, v):
        v = sympify(v)
        obj = Boolean.__new__(cls)
        obj._v = v
        return obj

    @property
    def v(self):
        return self._v

    @property
    def args(self):
        return (self._v,)

    def _hashable_content(self):
        return (self._v,)

    @property
    def free_symbols(self):
        return {self._v}

def _tau(v):
    return Tau(v)

def _vars(e):
    return set(e.free_symbols)

# ---------------------------
# Shannon expansion (NO simplify)
# ---------------------------
def shannon_expansion(f, s):
    """
    f == (s ∧ f|s=True) ∨ (¬s ∧ f|s=False)
    """
    if s not in f.free_symbols:
        return f
    f1 = f.subs({s: S.true})
    f0 = f.subs({s: S.false})
    return Or(And(s, f1), And(Not(s), f0))

# ---------------------------
# Utilities: componenti variabile-connesse
# ---------------------------
def _var_components(conj):
    """
    Raggruppa i fattori di un AND in blocchi variabile-connessi.
    Blocchi diversi non condividono variabili -> AND decomposable.
    """
    if isinstance(conj, And):
        factors = list(conj.args)
    else:
        factors = [conj]
    comps = []
    while factors:
        seed = factors.pop()
        group = [seed]
        vars_grp = set(seed.free_symbols)
        changed = True
        while changed:
            changed = False
            rest = []
            for f in factors:
                if vars_grp & set(f.free_symbols):
                    group.append(f)
                    vars_grp |= set(f.free_symbols)
                    changed = True
                else:
                    rest.append(f)
            factors = rest
        comps.append(group)
    return comps

# ---------------------------
# 1) NNF -> DNNF
# ---------------------------
def NNF2DNNF(nnf):
    """
      Function that takes a NNF and returns a DNNF.
      It applies Shannon's Expansion in order to ensure decomposability.
      Every conjunction does not have common atomic propositions, i.e. Φ₁ ∧ Φ₂ such that props(Φ₁) ∩ (Φ₂) = ∅.
    """
    #prendo le variabili booleane della formula e le inserisco in una lista
    #nnf.free_symbols è un set di Symbol di SymPy che compaiono nell’espressione
    atoms = list(nnf.free_symbols)

    #caso base: la formula ha 0 o 1 variabile
    if len(atoms) <= 1:
        return simplify(nnf)

    #costruisco gruppi che sono "variabili-connessi": se ho un solo gruppo → non decomponibile ⇒ applico Shannon
    if isinstance(nnf, And): #check se nodo corrente è congiunzione
        #_var_components prende i fattori dell’AND (es. F1 ∧ F2 ∧ F3) e li raggruppa in blocchi tali che dentro ogni blocco i fattori condividono variabili
        #se blocchi diversi non condividono variabili → sono candidati a un AND decomposable.
        #esempio: (A∨B)∧(¬A∨C)∧(D∨E) produce groups = [[A∨B, ¬A∨C], [D∨E]].
        groups = _var_components(nnf)
        if len(groups) > 1:
            compiled = [
                NNF2DNNF(And(*g)) if len(g) > 1 else NNF2DNNF(g[0])
                for g in groups
            ]
            return simplify(And(*compiled))  # AND decomposable
        else:
            # nessuna decomposizione possibile -> forza split con Shannon
            pivot = atoms[0]
            return NNF2DNNF(shannon_expansion(nnf, pivot))

    if isinstance(nnf, Or):
        kids = [NNF2DNNF(k) for k in nnf.args]
        return simplify(Or(*kids))          # determinism dopo

    # fallback
    pivot = atoms[0]
    return NNF2DNNF(shannon_expansion(nnf, pivot))

# ---------------------------
# 2) DNNF -> d-DNNF (determinism sugli OR) — versione robusta/iterativa
# ---------------------------
def _make_or_deterministic(expr):
    """
    Rende Or(...) deterministico senza ricorsioni infinite.
    Strategia:
      - finché esiste una coppia di figli congiungibile,
        scegli un pivot nella INTERSEZIONE delle loro variabili (se vuota, nell'UNIONE),
        applica Shannon all'intero OR e riparti. Iterativo.
    """
    assert isinstance(expr, Or)
    # flatten + dedup
    args = list(Or(*expr.args).args)
    uniq = []
    for a in args:
        if a not in uniq:
            uniq.append(a)
    args = uniq
    cur = Or(*args)

    while True:
        n = len(args)
        changed = False

        for i in range(n):
            for j in range(i+1, n):
                if satisfiable(And(args[i], args[j])):  # overlap -> non deterministico
                    Vi, Vj = _vars(args[i]), _vars(args[j])
                    cand = list(Vi & Vj) or list(Vi | Vj)  # prima intersezione, poi unione
                    for p in cand:
                        new_or = shannon_expansion(Or(*args), p)
                        if new_or != cur:       # verifica progresso strutturale
                            cur = new_or
                            args = list(cur.args) if isinstance(cur, Or) else [cur]
                            changed = True
                            break
                    break
            if changed:
                break
        if not changed:
            return cur

def DNNF2dDNNF(dnnf):
    if isinstance(dnnf, And):
        return And(*(DNNF2dDNNF(a) for a in dnnf.args))
    if isinstance(dnnf, Or):
        kids = [DNNF2dDNNF(a) for a in dnnf.args]
        det = _make_or_deterministic(Or(*kids))
        if isinstance(det, Or):
            return Or(*(DNNF2dDNNF(a) for a in det.args))
        return det
    return dnnf  # literal / True / False

# ---------------------------
# 3) d-DNNF -> sd-DNNF (smoothing strutturale con Tau)
# ---------------------------
def ddnnf2sdNNF(ddnnf):
    if isinstance(ddnnf, And):
        return And(*(ddnnf2sdNNF(a) for a in ddnnf.args))
    if isinstance(ddnnf, Or):
        kids = [ddnnf2sdNNF(a) for a in ddnnf.args]
        allv = set().union(*(_vars(k) for k in kids))
        smoothed = []
        for k in kids:
            missing = allv - _vars(k)
            kk = k
            for v in missing:
                kk = And(kk, _tau(v))  # mantieni Tau(v) visibile
            smoothed.append(kk)
        return Or(*smoothed)
    return ddnnf  # literal / True / False

# ---------------------------
# 4) WMC su sd-DNNF (pesi ai letterali)
# ---------------------------
def model_counting_sdnnf(sdNNF_formula, weights):
    """
    Calcola il WMC su una sd-DNNF:
      - literal: weights[literal]
      - Tau(v):  weights[v] + weights[Not(v)]
      - AND:     prodotto dei figli
      - OR:      somma dei figli
    """
    if sdNNF_formula is S.true:
        return 1.0
    if sdNNF_formula is S.false:
        return 0.0

    # letterali (a, ~a, ...)
    if sdNNF_formula in weights:
        return float(weights[sdNNF_formula])

    # Tau(v) inserita dallo smoothing
    if isinstance(sdNNF_formula, Tau):
        v = sdNNF_formula.v
        return float(weights[v] + weights[Not(v)])

    # AND: prodotto
    if isinstance(sdNNF_formula, And):
        val = 1.0
        for ch in sdNNF_formula.args:
            val *= model_counting_sdnnf(ch, weights)
        return val

    # OR: somma
    if isinstance(sdNNF_formula, Or):
        return sum(model_counting_sdnnf(ch, weights) for ch in sdNNF_formula.args)

    raise ValueError(f"Nodo non gestito: {sdNNF_formula}")

# ---------------------------
# 5) End-to-end: F -> sd-DNNF
# ---------------------------
def compile_to_sdDNNF(F):
    """
    F -> NNF -> DNNF -> d-DNNF -> sd-DNNF
    (niente simplify finale per non perdere i nodi Tau)
    """
    F_nnf  = to_nnf(F, simplify=True)   # rimuove ⇒, ↔ e spinge NOT ai letterali
    F_dnnf = NNF2DNNF(F_nnf)
    F_dd   = DNNF2dDNNF(F_dnnf)
    F_sdd  = ddnnf2sdNNF(F_dd)
    return F_sdd

per altri test case -> modifica sezione formula e pesi

In [3]:
# ===========================
# TEST CASE 1 (Example 7.5 dispensa - result = 136)
# ===========================
from sympy import symbols, And, Or, Not

# Formula: (a & b) | (c & ¬a)
a, b, c = symbols('a b c')
F = Or(And(a, b), And(c, Not(a)))
print("Formula:", F)

# Pesi
weights = {
    a: 2,   Not(a): 1,
    b: 5,   Not(b): 3,
    c: 7,   Not(c): 1
}

# Compilazione fino a sd-DNNF
F_sdd = compile_to_sdDNNF(F)
print("\nsd-DNNF:", F_sdd)

# Calcolo WMC con pesi reali
wmc_val = model_counting_sdnnf(F_sdd, weights)
print("\nWMC:", wmc_val)

Formula: (a & b) | (c & ~a)

sd-DNNF: (a & b & Tau(c)) | (c & ~a & Tau(b))

WMC: 136.0


In [4]:
# ===========================
# TEST CASE 2 — (Exercise 119 dispensa - result = 23)
# ===========================

from sympy import symbols, Or, Not, Implies

# Formula: (A ∨ B) → (B ∨ C)
A, B, C = symbols('A B C')
F = Implies(Or(A, B), Or(B, C))
print("Formula:", F)

# Pesi dei letterali (dallo screenshot)
weights = {
    A: 1,  Not(A): 2,
    B: 1,  Not(B): 2,
    C: 1,  Not(C): 2,
}

# Compilazione e WMC
F_sdd = compile_to_sdDNNF(F)
print("\nsd-DNNF:", F_sdd)

W = model_counting_sdnnf(F_sdd, weights)
print("WMC(F):", W)

Formula: Implies(A | B, B | C)

sd-DNNF: (B & Tau(A) & Tau(C)) | (~B & ((A & C) | (~A & Tau(C))))
WMC(F): 23.0
